In [2]:
import pyphonetics 
from pyphonetics import Soundex
from pyphonetics import Metaphone
from pyphonetics import RefinedSoundex
import pyjarowinkler
from pyjarowinkler import distance
import spacy
import codecs
import pprint
import metaphone
from metaphone import doublemetaphone
import phonetics
import pandas as pd


def LD(s,t):
    import pprint
    if len(s)==0 :
        res = len(t)
    if len(t)==0:
        res = len(s)
    dist = [[0 for x in range(len(s)+1)] for x in range(len(t)+1)]
    for i in range(len(s)+1):
        dist[0][i] = i
    for j in range(len(t)+1):
        dist[j][0] = j
    for i in range(1,len(s)+1):
        for j in range(1,len(t)+1):
            if s[i-1]==t[j-1]:
                cost = 0
            else:
                cost = 1
            dist[j][i] = min(dist[j-1][i]+1,dist[j][i-1]+1,dist[j-1][i-1]+cost)
    res = dist[len(t)][len(s)]
    #pprint.pprint(dist)
    return res

def Similarity_DL(s,t):
    return LD(s,t) / max(len(s),len(t))

def Distance_Jaro(a,b):
    return 1-distance.get_jaro_distance(a, b, winkler=True, scaling=0.1)

def Similarity_Word_Vectors(a,b):
    nlp = spacy.load('en_core_web_lg')
    a1 = nlp.vocab[a]
    b1 = nlp.vocab[b]
    indice = a1.similarity(b1)
    return indice

def Distance_Phonetics_Representations(a,b):
    rs = RefinedSoundex()
    if len(a)==len(b):
        return rs.distance(a,b,metric="hamming")
    else:
        return rs.distance(a,b,metric="levenshtein")

def Code_Soundex(a):
    soundex = Soundex()
    return soundex.phonetics(a)

def Comparaison_Soundex(a,b):
    soundex = Soundex()
    return soundex.sounds_like(a,b)

def Similarity_Soundex(a,b):
    return Distance_Jaro(Code_Soundex(a),Code_Soundex(b))

def Code_Metaphone(a):
    metaphone = Metaphone()
    return metaphone.phonetics(a)

def Similarity_Metaphone(a,b):
    return Distance_Jaro(Code_Metaphone(a),Code_Metaphone(b))

def Double_Metaphone(a):
    return str(phonetics.dmetaphone(a))

def ConstruireHTML(liste):
    w = open("20000-30000.html", "w")
    w.write("<table border= '1'>")
    for ligne in liste:
        w.write("<tr><td>")
        w.write("</td><td>".join([str(x) for x in ligne]))
        w.write("</td><tr>")
    w.write("</table>")
    w.close()

def Trouver_Paire_Minimale(t):# t est un tuple
    for i in range(len(t[0])):
        if(t[0][i]!=t[1][i]):
            return tuple((t[0][i],t[1][i]))

def stockerEnListe(a):
    s = []
    for couple in a:
        s.append(couple)
    return s

In [2]:
print(Double_Metaphone("pierre"))
print(Double_Metaphone("bière"))
print(Double_Metaphone("tier"))
print(Double_Metaphone("bière"))
print(Double_Metaphone("Pierre"))
print(Double_Metaphone("frotter"))
print(Double_Metaphone("frôler"))

('PR', '')
('PR', '')
('T', 'TR')
('PR', '')
('PR', '')
('FRTR', '')
('FRLR', '')


### http://redac.univ-tlse2.fr/lexiques/glaff.html

lien pour le dictionnaire GLAFF

### Prétraitement du dictionnaire

In [3]:
Mots = []
API = []
lignes = []
traits = []
lemme = []
Frequence = []

fin = codecs.open("GLAFF/glaff-1.2.2.txt",'r',encoding="UTF-8")
for line in open("GLAFF/glaff-1.2.2.txt",'r',encoding="UTF-8"):
    line = fin.readline()
    lignes.append(line)
fin.close()

for ligne in lignes:
    l = ligne.strip().split("|")
    if((l[3]!="") and (set(l[-12:-1])!={'0'})):
        Mots.append(l[0])
        traits.append(l[1])
        lemme.append(l[2])
        l[3]=l[3].replace('.','')
        API.append(l[3])

print(len(Mots))
print(len(traits))
print(len(lemme))
print(len(API))

758732
758732
758732
758732


### Problème 1: Pour les morphologies flexionnelles on ne peux pas lier directement le lemme et la transcription phonétique

http://paginaspersonales.deusto.es/abaitua/konzeptu/multext/LEX1_FR.html

###### Trouver les formes en masculin singulier

###### Diminuer la complexité

In [5]:
Mots_API = list(zip(Mots,API))
lemme_traits = list(zip(lemme,traits))
Mots_API_lemme_traits = list(zip(Mots_API,lemme_traits))
M = []
L = []
T = []
A = []
print(Mots_API_lemme_traits[0])
Information = dict()
for i in range(len(Mots_API_lemme_traits)):
    #Pour les verbes:
    if(Mots_API_lemme_traits[i][1][1]=="Vmn----"):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Verbe")
    # Pour les noms communs/propres:
    if(Mots_API_lemme_traits[i][1][1]=="Ncms"):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Nom")
    # Pour les adjectifs:
    if((Mots_API_lemme_traits[i][1][1]=="Afcms-")or(Mots_API_lemme_traits[i][1][1]=="Afpms-")or(Mots_API_lemme_traits[i][1][1]=="Ai-ms-")or(Mots_API_lemme_traits[i][1][1]=="Ao-ms-")or(Mots_API_lemme_traits[i][1][1]=="As-ms-")):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Adjectif")
    # Pour les adverbes:
    if((Mots_API_lemme_traits[i][1][1]=="Rgp")or(Mots_API_lemme_traits[i][1][1]=="Rgc")or(Mots_API_lemme_traits[i][1][1]=="Rgc")or(Mots_API_lemme_traits[i][1][1]=="Rgn")):
        M.append(Mots_API_lemme_traits[i][1][0])
        A.append(Mots_API_lemme_traits[i][0][1])
        L.append(Mots_API_lemme_traits[i][1][0])
        T.append("Adverbe") 

(('übercélèbre', 'ybœʁselɛbʁ'), ('übercélèbre', 'Afpfs'))


In [5]:
with open("CorpusNet.txt",'w',encoding="UTF-8")as f:
    for i in range(len(M)):
        s = M[i]+"|"+L[i]+"|"+T[i]+"|"+A[i]
        f.write(s)
        f.write("\n")
with open("Corpus.txt",'w',encoding="UTF-8")as f:
    for i in range(len(M)):
        s = L[i]+"|"+T[i]+"|"+A[i]
        f.write(s)
        f.write("\n")

### Problème 2 Trouver les paires-minimales

In [8]:
print(len(M))
print(len(A))
print(len(T))
print(len(L))
Corpus = list(zip(list(zip(M,A)),T))
print(len(Corpus))

37580
37580
37580
37580
37580


In [11]:
# 10000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales = []
Paires_Minimales_API = []
Paires_Minimales_POS = []
for i in range(len(M[:10000])):
    for j in range(i+1,len(M[:10000])):
        if((LD(Corpus[i][0][1],Corpus[j][0][1])==1) and (len(Corpus[i][0][1])==len(Corpus[j][0][1]))):
            Paires_Minimales.append(tuple((Corpus[i][0][0],Corpus[j][0][0])))
            Paires_Minimales_API.append(tuple((Corpus[i][0][1],Corpus[j][0][1])))
            Paires_Minimales_POS.append(tuple((Corpus[i][1],Corpus[j][1])))
print(len(Paires_Minimales))
print(len(Paires_Minimales_API))
print(len(Paires_Minimales_POS))
print(Paires_Minimales[211],Paires_Minimales_API[211],Paires_Minimales_POS[211])

7065
7065
7065
('about', 'adoux') ('abu', 'adu') ('Nom', 'Nom')


In [12]:
# 20000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales1 = []
Paires_Minimales_API1 = []
Paires_Minimales_POS1 = []
for i in range(10000,20000):
    for j in range(i+1,20000):
        if((LD(Corpus[i][0][1],Corpus[j][0][1])==1) and (len(Corpus[i][0][1])==len(Corpus[j][0][1]))):
            Paires_Minimales1.append(tuple((Corpus[i][0][0],Corpus[j][0][0])))
            Paires_Minimales_API1.append(tuple((Corpus[i][0][1],Corpus[j][0][1])))
            Paires_Minimales_POS1.append(tuple((Corpus[i][1],Corpus[j][1])))
print(len(Paires_Minimales_API1))
print(len(Paires_Minimales_POS1))
print(len(Paires_Minimales1))
print(Paires_Minimales1[211],Paires_Minimales_API1[211],Paires_Minimales_POS1[211])

5233
5233
5233
('croton', 'crottin') ('kʁɔtɔ̃', 'kʁɔtɛ̃') ('Nom', 'Nom')


In [55]:
# 30000
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales2 = []
Paires_Minimales_API2 = []
Paires_Minimales_POS2 = []
for i in range(20000,30000):
    for j in range(i+1,30000):
        if((LD(Corpus[i][0][1],Corpus[j][0][1])==1) and (len(Corpus[i][0][1])==len(Corpus[j][0][1]))):
            Paires_Minimales2.append(tuple((Corpus[i][0][0],Corpus[j][0][0])))
            Paires_Minimales_API2.append(tuple((Corpus[i][0][1],Corpus[j][0][1])))
            Paires_Minimales_POS2.append(tuple((Corpus[i][1],Corpus[j][1])))
print(len(Paires_Minimales_API2))
print(len(Paires_Minimales_POS2))
print(len(Paires_Minimales2))
print(Paires_Minimales2[211],Paires_Minimales_API2[211],Paires_Minimales_POS2[211])

6489
6489
6489
('kinnor', 'quinoa') ('kinɔʁ', 'kinɔa') ('Nom', 'Nom')


In [10]:
# 37580
Corpus = list(zip(list(zip(M,A)),T))
Paires_Minimales3 = []
Paires_Minimales_API3 = []
Paires_Minimales_POS3 = []
for i in range(30000,37580):
    for j in range(i+1,37580):
        if((LD(Corpus[i][0][1],Corpus[j][0][1])==1) and (len(Corpus[i][0][1])==len(Corpus[j][0][1]))):
            Paires_Minimales3.append(tuple((Corpus[i][0][0],Corpus[j][0][0])))
            Paires_Minimales_API3.append(tuple((Corpus[i][0][1],Corpus[j][0][1])))
            Paires_Minimales_POS3.append(tuple((Corpus[i][1],Corpus[j][1])))
print(len(Paires_Minimales_API3))
print(len(Paires_Minimales_POS3))
print(len(Paires_Minimales3))
print(Paires_Minimales3[211],Paires_Minimales_API3[211],Paires_Minimales_POS3[211])

4668
4668
4668
('regret', 'retrait') ('ʁəgʁɛ', 'ʁətʁɛ') ('Nom', 'Nom')


In [7]:
def Trouver_Paire_Minimale(t):# t est un tuple
    for i in range(len(t[0])):
        if(t[0][i]!=t[1][i]):
            return tuple((t[0][i],t[1][i]))
print(Paires_Minimales_API[1311][0])
print(Trouver_Paire_Minimale(Paires_Minimales_API[1311]))

amɔliʁ
('m', 's')


### 3. Stocker les données et Construire le tableau

In [56]:
Mots10000 = stockerEnListe(Paires_Minimales)
APIs10000 = stockerEnListe(Paires_Minimales_API)
Traits10000 = stockerEnListe(Paires_Minimales_POS)
Mots1000020000 = stockerEnListe(Paires_Minimales1)
APIs1000020000 = stockerEnListe(Paires_Minimales_API1)
Traits1000020000 = stockerEnListe(Paires_Minimales_POS1)
Mots2000030000 = stockerEnListe(Paires_Minimales2)
APIs2000030000 = stockerEnListe(Paires_Minimales_API2)
Traits2000030000 = stockerEnListe(Paires_Minimales_POS2)
Mots3000037580 = stockerEnListe(Paires_Minimales3)
APIs3000037580 = stockerEnListe(Paires_Minimales_API3)
Traits3000037580 = stockerEnListe(Paires_Minimales_POS3)

In [59]:
with open("PM2000030000APIs.txt",'w',encoding = "UTF-8")as f:
    for ligne in APIs2000030000:
        f.write(ligne)
        f.write("\n")

In [17]:
Tableau = [[[]for x in range(5)]for y in range(len(Paires_Minimales)+1)]
Tableau[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales)):
    Tableau[i+1][0]=str(Paires_Minimales[i])
    Tableau[i+1][1]=str(Paires_Minimales_API[i])
    Tableau[i+1][2]=str(Paires_Minimales_POS[i])
    Tableau[i+1][3]=Double_Metaphone(Paires_Minimales[i][0])+" "+Double_Metaphone(Paires_Minimales[i][1])
    Tableau[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API[i]))
ConstruireHTML(Tableau)

In [19]:
Tableau1 = [[[]for x in range(5)]for y in range(len(Paires_Minimales1)+1)]
Tableau1[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales1)):
    Tableau1[i+1][0]=str(Paires_Minimales1[i])
    Tableau1[i+1][1]=str(Paires_Minimales_API1[i])
    Tableau1[i+1][2]=str(Paires_Minimales_POS1[i])
    Tableau1[i+1][3]=Double_Metaphone(Paires_Minimales1[i][0])+" "+Double_Metaphone(Paires_Minimales1[i][1])
    Tableau1[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API1[i]))
ConstruireHTML(Tableau1)

In [60]:
Tableau2 = [[[]for x in range(5)]for y in range(len(Paires_Minimales2)+1)]
Tableau2[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales2)):
    Tableau2[i+1][0]=str(Paires_Minimales2[i])
    Tableau2[i+1][1]=str(Paires_Minimales_API2[i])
    Tableau2[i+1][2]=str(Paires_Minimales_POS2[i])
    Tableau2[i+1][3]=Double_Metaphone(Paires_Minimales2[i][0])+" "+Double_Metaphone(Paires_Minimales2[i][1])
    Tableau2[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API2[i]))
ConstruireHTML(Tableau2)

In [21]:
Tableau3 = [[[]for x in range(5)]for y in range(len(Paires_Minimales3)+1)]
Tableau3[0] = ["Mots","Transcription Phonétique","POS","Double Métaphone","Paires Minimales"]
for i in range(len(Paires_Minimales3)):
    Tableau3[i+1][0]=str(Paires_Minimales3[i])
    Tableau3[i+1][1]=str(Paires_Minimales_API3[i])
    Tableau3[i+1][2]=str(Paires_Minimales_POS3[i])
    Tableau3[i+1][3]=Double_Metaphone(Paires_Minimales3[i][0])+" "+Double_Metaphone(Paires_Minimales3[i][1])
    Tableau3[i+1][4]=str(Trouver_Paire_Minimale(Paires_Minimales_API3[i]))
ConstruireHTML(Tableau3)

In [20]:
with open("Paires Minimales POS 10000.txt",'w',encoding="UTF-8")as f:
    for ligne in Paires_Minimales_POS:
        f.write(str(ligne))
        f.write('\n')

In [2]:
import tools
import codecs
import pprint
f = codecs.open("glaffwords.txt",'r',"utf-8")
words = []
for i in range(10336):
    g=f.readline()
    l=g.split("|")
    l[-1] = l[-1].replace("\n","")
    words.append(l)
f.close()
# Pour trouver le vocabulaire 
words_with_API = []
words_without_API = []
for word in words:
    if len(word)!=4:
        words_without_API.append(word)
    else:
        words_with_API.append(word)
# créer un dictionaire qui lie le mot(supprimer les copies) avec leur API
Vocabulaire_Informations = dict()
for word in words_with_API:
    if ";" in word[3]:
        Vocabulaire_Informations[word[2]]=word[3][:word[3].find(";")]# Il y a un problème de choix(1/2 pour la transcription ;) que je dois vous dire ce vendredi 
    else:
        Vocabulaire_Informations[word[2]]=word[3]
# créer une liste de vocabulaire (2143 mots différents)
Vocabulaire = []
API = []
for word,prononciation in Vocabulaire_Informations.items():
    Vocabulaire.append(word)
    API.append(prononciation)
# créer des samples qui contienent 10 paires de mots différents.Et leur API concerné
Sample1 = list(zip(Vocabulaire[:10],Vocabulaire[10:20]))
APISample1 = list(zip(API[:10],API[10:20]))
Sample2 = list(zip(Vocabulaire[100:110],Vocabulaire[110:120]))
APISample2 = list(zip(API[100:110],API[110:120]))
Sample3 = list(zip(Vocabulaire[1000:1010],Vocabulaire[2000:2010]))
APISample3 = list(zip(API[100:110],API[110:120]))
# Il y a beaucoup d'autres samples, ici la taille d'un sample est de 10 paires de mots.
# un tableaux de 11 x 10 qui qffichie tous les indices concernant les sons de deux mots choisis.
def Tableaux_de_Résultats(Sample,APISample):
    Tableaux = [[[]for x in range(10)]for y in range(11)]
    T = [["Mots:"],["API"],["Soundex"],["Similarity_Soundex"],["Alike?"],["Metaphone"],["Similarity_Metaphone"],["Distance_Levenshtein"],["Similarity Distance Levenshtein"],["Jaro Distance"]]
    Tableaux[0]=T
    for i in range(1,11):
        Tableaux[i][0]=Sample[i-1]
        Tableaux[i][1]=APISample[i-1]
        Tableaux[i][2]=(tools.Code_Soundex(Sample[i-1][0]),tools.Code_Soundex(Sample[i-1][1]))
        Tableaux[i][3]=tools.Similarity_Soundex(Sample[i-1][0],Sample[i-1][1])
        Tableaux[i][4]=tools.Comparaison_Soundex(Sample[i-1][0],Sample[i-1][1])
        Tableaux[i][5]=(tools.Code_Metaphone(Sample[i-1][0]),tools.Code_Metaphone(Sample[i-1][1]))
        Tableaux[i][6]=tools.Similarity_Metaphone(Sample[i-1][0],tools.Code_Metaphone(Sample[i-1][1]))
        Tableaux[i][7]=tools.LD(APISample[i-1][0],APISample[i-1][1])
        Tableaux[i][8]=tools.Similarity_DL(APISample[i-1][0],APISample[i-1][1])
        Tableaux[i][9]=tools.Distance_Jaro(APISample[i-1][0],APISample[i-1][1])
    return Tableaux
pprint.pprint(Tableaux_de_Résultats(Sample1,APISample1))
pprint.pprint(Tableaux_de_Résultats(Sample2,APISample2))
import json
j = json.dumps(Tableaux_de_Résultats(Sample2,APISample2),ensure_ascii=False)
with codecs.open("Résultat.json",'w',"utf-8")as fin:
    fin.write(j)

FileNotFoundError: [Errno 2] No such file or directory: 'glaffwords.txt'